In [7]:
!pip install mlflow

     ---------------------------------------- 20.2/20.2 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 56.7/56.7 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 145.3/145.3 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 10.6/10.6 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 147.6/147.6 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 3.3 MB/s eta 0:00:00
     -------------------------------------- 207.3/207.3 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 128.2/128.2 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 133.3/133.3 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 24.8/24.8 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 233.4/233.4 kB 3.5 MB/s eta 0:00:00
     -------------------------------------


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [13]:
import os
import sys

# Añadimos la carpeta 'drive' al path
ruta_carpeta_drive = os.path.abspath('../drive') #.. era para salirse de la carpeta actual y entrar en la de drive
if ruta_carpeta_drive not in sys.path:
    sys.path.insert(0, ruta_carpeta_drive)

import drive

# Descargamos los datos en formato parquet de Google Drive
# guardar localmente
drive.descargar_archivos_concretos('DATOS_MAX.parquet', '../drive')
#hay que pasarle la ruta a la carpeta drive desde donde lo estamos ejecutando
# y el nombre del archivo que hemos puesto en el archivos_info.txt (MLP_runs.db en nuestro ejemplo, en vez de datosEntrenamiento.parquet...)
# leer para utilizar esto ya en el modelos
data = pd.read_parquet('datos/DATOS_MAX.parquet') # 

Archivo DATOS_MAX.parquet guardado en: ./datos\DATOS_MAX.parquet


In [15]:
data

,name,EventStartTime,SalesStartTIme,SalesEndTime,max_price,nameArtist,VenueName,VenueCity,VenueState,num_links,...,Rock,Singer-Songwriter,Soul,Urban,World,tiene_wiki,tiene_youtube,dias_Venta,dias_desdeFinVenta,num_noches
0,Jamie Cullum,2024-06-14 20:00:00+00:00,2023-11-30 16:56:00+00:00,2024-06-12 21:59:00+00:00,59.29,Jamie Cullum,Auditorio Starlite,Marbella,Málaga,9,...,0,0,0,0,0,1,1,195,1,1
1,Sheryl Crow,2024-06-21 20:00:00+00:00,2023-11-28 20:57:00+00:00,2024-06-19 21:59:00+00:00,80.00,Sheryl Crow,Auditorio Starlite,Marbella,Málaga,9,...,1,0,0,0,0,1,1,204,1,1
2,Aitana,2024-06-22 20:00:00+00:00,2024-01-09 12:29:00+00:00,2024-06-20 21:59:00+00:00,146.43,Aitana,Auditorio Starlite,Marbella,Málaga,5,...,0,0,0,0,0,0,1,163,1,1
3,Christian Nodal,2024-07-13 20:00:00+00:00,2023-11-28 21:00:00+00:00,2024-07-11 21:59:00+00:00,79.29,Christian Nodal,Auditorio Starlite,Marbella,Málaga,8,...,0,0,0,0,1,0,1,226,1,1
4,Take That,2024-07-14 20:00:00+00:00,2023-11-28 21:00:00+00:00,2024-07-12 21:59:00+00:00,155.36,Robbie Williams,Auditorio Starlite,Marbella,Málaga,10,...,1,0,0,0,0,1,1,227,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Abba The New Experience & Queen Forever,2024-05-29 18:00:00+00:00,2024-02-28 14:00:00+00:00,2024-05-29 18:00:00+00:00,20.00,Abba The New Experience,Zona Camping Circuit Catalunya,Montmeló,Barcelona,0,...,1,0,0,0,0,0,0,91,0,1
490,Front 242,2024-10-13 18:30:00+00:00,2024-03-08 09:00:00+00:00,2024-10-13 18:30:00+00:00,33.13,Front 242,Sala Apolo,Barcelona,Barcelona,9,...,1,0,0,0,0,1,1,219,0,1
491,Front 242,2024-10-14 19:00:00+00:00,2024-03-08 09:00:00+00:00,2024-10-14 19:00:00+00:00,33.13,Front 242,Sala La Paqui (Antigua Sala But),Madrid,Madrid,9,...,1,0,0,0,0,1,1,220,0,1
492,Garbage,2024-10-16 18:30:00+00:00,2024-03-08 09:00:00+00:00,2024-10-16 18:30:00+00:00,40.00,Garbage,Sala Razzmatazz 1,Barcelona,Barcelona,10,...,1,0,0,0,0,1,1,222,0,1


data

In [ ]:
#import pandas as pd

# Leer el archivo CSV utilizando una ruta relativa
#data = pd.read_csv("/Users/yushanyang/Desktop/proyecto_aa/ data/ df_predecir_max.csv")
#data

,name,EventStartTime,SalesStartTIme,SalesEndTime,max_price,nameArtist,VenueName,VenueCity,VenueState,num_links,...,Rock,Singer-Songwriter,Soul,Urban,World,tiene_wiki,tiene_youtube,dias_Venta,dias_desdeFinVenta,num_noches
0,Jamie Cullum,2024-06-14 20:00:00+00:00,2023-11-30 16:56:00+00:00,2024-06-12 21:59:00+00:00,59.29,Jamie Cullum,Auditorio Starlite,Marbella,Málaga,9,...,0,0,0,0,0,1,1,195,1,1
1,Sheryl Crow,2024-06-21 20:00:00+00:00,2023-11-28 20:57:00+00:00,2024-06-19 21:59:00+00:00,80.00,Sheryl Crow,Auditorio Starlite,Marbella,Málaga,9,...,1,0,0,0,0,1,1,204,1,1
2,Aitana,2024-06-22 20:00:00+00:00,2024-01-09 12:29:00+00:00,2024-06-20 21:59:00+00:00,146.43,Aitana,Auditorio Starlite,Marbella,Málaga,5,...,0,0,0,0,0,0,1,163,1,1
3,Christian Nodal,2024-07-13 20:00:00+00:00,2023-11-28 21:00:00+00:00,2024-07-11 21:59:00+00:00,79.29,Christian Nodal,Auditorio Starlite,Marbella,Málaga,8,...,0,0,0,0,1,0,1,226,1,1
4,Take That,2024-07-14 20:00:00+00:00,2023-11-28 21:00:00+00:00,2024-07-12 21:59:00+00:00,155.36,Robbie Williams,Auditorio Starlite,Marbella,Málaga,10,...,1,0,0,0,0,1,1,227,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Abba The New Experience & Queen Forever,2024-05-29 18:00:00+00:00,2024-02-28 14:00:00+00:00,2024-05-29 18:00:00+00:00,20.00,Abba The New Experience,Zona Camping Circuit Catalunya,Montmeló,Barcelona,0,...,1,0,0,0,0,0,0,91,0,1
490,Front 242,2024-10-13 18:30:00+00:00,2024-03-08 09:00:00+00:00,2024-10-13 18:30:00+00:00,33.13,Front 242,Sala Apolo,Barcelona,Barcelona,9,...,1,0,0,0,0,1,1,219,0,1
491,Front 242,2024-10-14 19:00:00+00:00,2024-03-08 09:00:00+00:00,2024-10-14 19:00:00+00:00,33.13,Front 242,Sala La Paqui (Antigua Sala But),Madrid,Madrid,9,...,1,0,0,0,0,1,1,220,0,1
492,Garbage,2024-10-16 18:30:00+00:00,2024-03-08 09:00:00+00:00,2024-10-16 18:30:00+00:00,40.00,Garbage,Sala Razzmatazz 1,Barcelona,Barcelona,10,...,1,0,0,0,0,1,1,222,0,1


In [ ]:
# Sets the sqlite db as the MLFLOW_TRACKING_URI 
import os
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# WARNING: TO SEE THE LOCAL SERVER YOU HAVE TO CHOOSE THE CORRECT BACKEND STORE AS FOLLOWS:
# mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [ ]:
mlflow.set_experiment("MLflow Prueba")

2024/05/06 17:33:10 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Prueba' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/536475883718580488', creation_time=1715009590405, experiment_id='536475883718580488', last_update_time=1715009590405, lifecycle_stage='active', name='MLflow Prueba', tags={}>

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Eliminamos las categoricas
columnas=['name','EventStartTime', 'SalesStartTIme', 'SalesEndTime', 'nameArtist', 'VenueName',
          'VenueCity', 'VenueState','Generos_combinados']

data.drop(columnas, axis=1, inplace=True)

RANDOM_STATE = 83 #fijamos la semilla


In [ ]:
# Dividir datos en conjuntos de entrenamiento y prueba
X = data.drop(columns=['max_price'])
y = data['max_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Crear datos de ejemplo
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)

# Hiperparámetros
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=1000, num=10)],
    'max_features': ['log2', 'sqrt', None],
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}


# Inicializar el modelo
rf = RandomForestRegressor()

# Inicializar RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=200, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Entrenar modelo con búsqueda aleatoria
random_search.fit(X, y)

# Obtener el mejor modelo de la búsqueda aleatoria
best_model = random_search.best_estimator_

# Predicciones con el mejor modelo
y_pred = best_model.predict(X)

# Calcular métricas
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

# Iniciar una nueva ejecución en MLflow
with mlflow.start_run(run_name='RandomForestRegressor'):

    # Log de parámetros
    mlflow.log_params(random_search.best_params_)

    # Log de métricas
    mlflow.log_metrics({
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R2": r2
    })

    # Log del modelo
    mlflow.sklearn.log_model(best_model, "random_forest_regressor_model")


Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.0s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.0s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.0s
[CV] END bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   2.0s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.0s
[CV] END bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   1.9s
[CV] END bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   2.0s
[CV] END bootstrap=True, max_depth=100

/Applications/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
